In [1]:
import time
import requests
import pandas as pd
import pypsa


In [2]:
# Initialize network
network = pypsa.Network()

In [6]:
#Add buses for generator & load
network.add("Bus", "generation")
network.add("Bus", "load")

#Add generator at generation bus
network.add("Generator", "gen", bus="generation", p_nom=100)

#Add load at load bus
network.add("Load", "load", bus="load", p_set=80)

#Add a transmission line between the buses
network.add("Line", "line", bus0="generation", bus1="load", p_nom=100)


Index(['line'], dtype='object')

In [7]:
#Run the power flow analysis
network.pf()
print(network.buses)

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network <pypsa.components.SubNetwork object at 0x159a4b1d0> for snapshots Index(['now'], dtype='object', name='snapshot')


            v_nom type    x    y carrier unit  v_mag_pu_set  v_mag_pu_min  \
Bus                                                                         
generation    1.0       0.0  0.0      AC                1.0           0.0   
load          1.0       0.0  0.0      AC                1.0           0.0   

            v_mag_pu_max control generator sub_network  
Bus                                                     
generation           inf   Slack       gen           0  
load                 inf      PQ      None           0  


NREL API DATA FOR JAN 2025 NYC

In [19]:
import requests
import time
import pandas as pd
import os

# Example API call
url = "https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv"
params = {
    "api_key": "vcrBDcmJu7vmFIPTOueMRxnb2UZD4YD6zsnZXq37",
    "lat": 40.7128,
    "lon": -74.0060,
    "start_date": "2025-01-01",
    "end_date": "2025-01-31",
    "variables": "ghi,dni",
    "timestep": "day"
}

response = requests.get(url, params=params)

# Handle rate limit if encountered (retry if rate limit is reached)
if response.status_code == 429:  # Rate limit error
    print("Rate limit reached, waiting 60 seconds before retrying.")
    time.sleep(60)
    response = requests.get(url, params=params)  # Retry the request

# Check if the response is successful (status code 200)
if response.status_code == 200:
    # Save data to CSV (optional)
    with open('solar_data.csv', 'wb') as file:
        file.write(response.content)
    print("Data successfully fetched and saved!")
else:
    print("Error fetching data:", response.status_code)

# Check if the file exists and is not empty before loading it into a Pandas DataFrame
if os.path.exists('solar_data.csv') and os.path.getsize('solar_data.csv') > 0:
    try:
        # Load the solar data into a Pandas DataFrame
        solar_data = pd.read_csv('solar_data.csv')
        print("Data successfully loaded into DataFrame!")
    except pd.errors.EmptyDataError:
        print("File 'solar_data.csv' is empty or has no columns to parse.")
else:
    print("File 'solar_data.csv' does not exist or is empty.")



Error fetching data: 400
File 'solar_data.csv' does not exist or is empty.
